<a href="https://colab.research.google.com/github/opeoje91/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,415 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,185 kB]
Hit:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-16 23:31:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-06-16 23:31:59 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
wireless_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
wireless_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

### Create DataFrames to match tables

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [8]:
# Create the customers_table DataFrame

customers_df = wireless_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")

#customers_df.show(3)

In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = wireless_df.select(["product_id", "product_title"]).drop_duplicates()
#products_df.show(3)

In [10]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = wireless_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
#review_id_df.show(3)
review_id_df.count()

9002021

In [11]:
review_id_small_df = review_id_df.sort('customer_id').limit(100)
review_id_small_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LBCLIG3P2292|      10005|B008BR5WHK|      33899787| 2015-08-13|
|R1TF7ZVNCDPM4Q|      10011|B00RAYU41O|     391014311| 2015-04-11|
|R3F70D1KUTUY8A|      10021|B00G4RKJX6|     496269204| 2014-09-07|
| R83BPYNV6WND5|      10045|B007C5S3AU|     236779215| 2013-02-23|
|R3IYPHKK48WQ5U|      10046|B00TKWEO9W|     352176523| 2015-06-19|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:

review_id_small_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LBCLIG3P2292|      10005|B008BR5WHK|      33899787| 2015-08-13|
|R1TF7ZVNCDPM4Q|      10011|B00RAYU41O|     391014311| 2015-04-11|
|R3F70D1KUTUY8A|      10021|B00G4RKJX6|     496269204| 2014-09-07|
| R83BPYNV6WND5|      10045|B007C5S3AU|     236779215| 2013-02-23|
|R17BH4DUZW5VLB|      10046|B00UD15SV2|     700819973| 2015-06-19|
+--------------+-----------+----------+--------------+-----------+



In [12]:
# Create the vine_table. DataFrame
vine_df = wireless_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
#vine_df.show(3)


In [15]:
vine_small_df = vine_df.sort("review_id").limit(100)
vine_small_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10000B5EA0ZWP|          4|           14|         15|   N|                Y|
|R10000KAJO8QRT|          5|            0|          0|   N|                Y|
|R10000SRLJQ0VM|          5|            0|          0|   N|                Y|
|R10000W0Z79FQC|          4|            0|          0|   N|                Y|
|R1000147G0K1IW|          5|            0|          0|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [16]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazondata.ceaygspn5nsf.us-east-2.rds.amazonaws.com:5432/amazondatadb"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [17]:
# Write review_id_df to table in RDS
review_id_small_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [19]:
# Write products_df to table in RDS
# about 3 min
#products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
# 5 min 14 s
#customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [22]:
# Write vine_df to table in RDS
# 11 minutes
#vine_small_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)